# 3. Funções Utilizadas

## 3.1. Análises gerais

In [ ]:
def verificar_duplicatas(df, col_id="msno", col_safra="safra", visualizar=False):
    """
    Identifica e conta linhas duplicadas para a mesma chave (ID + Safra).
    """
    # Define a janela abrangendo todas as linhas do grupo
    window_spec = Window.partitionBy(col_id, col_safra)\
                        .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    
    # Adiciona contagem e filtra duplicados
    df_duplicados = df.withColumn("contagem_janela", F.count("*").over(window_spec)).filter("contagem_janela > 1")
    
    if visualizar:
        df_duplicados.show(10)
    
    # Retorna a quantidade total de linhas duplicadas encontradas
    return df_duplicados.count()

In [ ]:
def verificar_mudanca_estado(df, col_alvo, col_id="msno", col_safra="safra", visualizar=False):
    """
    Verifica mudanças de estado em uma variável ao longo do tempo (safras).
    """
    # Definir a especificação da janela
    window_spec = Window.partitionBy(col_id).orderBy(col_safra)

    # Lag para criar as colunas de estado anterior da variavel e detecção de mudança
    df_historico = (
        df.withColumn(f"{col_alvo}_anterior", F.lag(col_alvo).over(window_spec))
          .withColumn("mudou", 
              F.when(F.col(f"{col_alvo}_anterior").isNull(), False)
               .otherwise(F.col(col_alvo) != F.col(f"{col_alvo}_anterior"))))

    # Caso o usuario deseje visualizar as mudancas
    if visualizar:
        # Mostrar quantas mudancas ocorreram por safra
        print("Contagem de mudanças por safra:")
        df_historico.filter(F.col("mudou") == True).groupBy(col_safra, "mudou").count().orderBy(col_safra, "mudou").show(30, truncate=False)
        # Mostrar quantas vezes o usuario mudou
        print("Mudanças por usuário:")
        df_historico.filter(F.col("mudou") == True).groupBy(col_id).agg(F.sum(F.col("mudou").cast("int")).alias("total_mudancas")).show(10, truncate=False)

In [ ]:
def calcular_distribuicao(df, colunas_alvo, col_id="msno", n_show=10, agrupar_por_safra=False):
    """
    Gera agregação de contagem e percentual sobre uma variável alvo.
    """
    # 1. Calcula o total da base para o percentual global
    total_base = df.count()

    if agrupar_por_safra:
        # 2. Agrupamento inicial por safra e alvo
        df_result = df.groupBy("safra", *colunas_alvo).agg(
            F.count(col_id).alias("total")
        )
        
        # 3. Define a janela para calcular o total por safra (denominador do pct_safra)
        window_safra = Window.partitionBy("safra")
        
        # 4. Adiciona os cálculos de percentual
        df_result = df_result.withColumn(
            "pct_safra", 
            F.round((F.col("total") / F.sum("total").over(window_safra)) * 100, 2)
        ).withColumn(
            "pct_total", 
            F.round((F.col("total") / F.lit(total_base)) * 100, 2)
        ).orderBy("safra", F.desc("total"))
        
    else:
        # Resultado geral sem partição de safra
        df_result = df.groupBy(colunas_alvo).agg(
            F.count(col_id).alias("total"),
            F.round((F.count(col_id) / F.lit(total_base)) * 100, 2).alias("pct_total")
        ).orderBy(F.desc("total"))
    
    df_result.show(n_show, truncate=False)
    return df_result # Retornar o DF é uma boa prática para análises posteriores

In [ ]:
def correlation_matrix(df, colunas, plot=True, top_n=15, figsize=(20, 8)):
    """
    Calcula e visualiza matriz de correlação com layout otimizado.
    
    Parâmetros:
    -----------
    df : Spark DataFrame
    colunas : Lista com nomes das colunas numéricas
    plot : bool - Se True, gera visualização
    top_n : int - Número máximo de features a exibir (ordenadas por variância ou IV)
    figsize : tuple - Tamanho da figura
    """
    n = len(colunas)
    
    # ========================================================================
    # CASO 1: Par de variáveis (Scatter Plot)
    # ========================================================================
    if n == 2:
        col1, col2 = colunas
        
        pearson_val = df.corr(col1, col2, method="pearson")
        spearman_val = df.corr(col1, col2, method="spearman")
        
        print(f"📊 Correlação de Pearson:  {pearson_val:+.4f}")
        print(f"📊 Correlação de Spearman: {spearman_val:+.4f}")

        if plot:
            amostra_pd = df.select(col1, col2).sample(
                False, 
                fraction=min(1.0, 50000/df.count())
            ).toPandas()
            
            fig, ax = plt.subplots(figsize=(10, 7))
            
            # Scatter com densidade
            scatter = ax.scatter(
                amostra_pd[col1], 
                amostra_pd[col2], 
                alpha=0.4, 
                s=20,
                c=amostra_pd[col1],
                cmap='viridis',
                edgecolors='none'
            )
            
            # Linha de tendência
            z = np.polyfit(amostra_pd[col1], amostra_pd[col2], 1)
            p = np.poly1d(z)
            ax.plot(
                amostra_pd[col1].sort_values(), 
                p(amostra_pd[col1].sort_values()), 
                "r--", 
                alpha=0.8, 
                linewidth=2,
                label=f'Tendência: y = {z[0]:.3f}x + {z[1]:.3f}'
            )
            
            ax.set_xlabel(col1, fontsize=12, fontweight='bold')
            ax.set_ylabel(col2, fontsize=12, fontweight='bold')
            ax.set_title(
                f"Dispersão: {col1} vs {col2}\n"
                f"Pearson: {pearson_val:+.3f}  |  Spearman: {spearman_val:+.3f}",
                fontsize=14,
                fontweight='bold',
                pad=20
            )
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.legend(loc='best', fontsize=10)
            
            plt.colorbar(scatter, ax=ax, label=col1)
            plt.tight_layout()
            plt.show()
            plt.close()
    
    # ========================================================================
    # CASO 2: Matriz de correlação (> 2 variáveis)
    # ========================================================================
    else:
        # Limitar número de features se necessário
        if n > top_n:
            print(f"⚠️  Muitas features ({n}). Exibindo apenas top {top_n} por variância.")
            
            # Calcular variância de cada coluna
            variancias = []
            for col in colunas:
                var = df.select(F.variance(col)).collect()[0][0]
                variancias.append((col, var if var else 0))
            
            # Ordenar por variância e pegar top_n
            colunas_top = [col for col, _ in sorted(variancias, key=lambda x: x[1], reverse=True)[:top_n]]
        else:
            colunas_top = colunas
        
        # Preparar vetor
        assembler = VectorAssembler(
            inputCols=colunas_top, 
            outputCol="features", 
            handleInvalid="skip"
        )
        df_vector = assembler.transform(df).select("features")
        
        # Calcular matrizes
        def get_corr_matrix(method):
            matrix = Correlation.corr(df_vector, "features", method=method).collect()[0][0]
            return pd.DataFrame(matrix.toArray(), index=colunas_top, columns=colunas_top)
        
        print("🔄 Calculando matriz de Pearson...")
        matrix_pearson = get_corr_matrix("pearson")
        
        print("🔄 Calculando matriz de Spearman...")
        matrix_spearman = get_corr_matrix("spearman")
        
        if plot:
            # ============================================================
            # VISUALIZAÇÃO OTIMIZADA
            # ============================================================
            
            # Truncar nomes longos para melhor legibilidade
            labels_curtos = [
                col[:30] + '...' if len(col) > 30 else col 
                for col in colunas_top
            ]
            
            fig, axes = plt.subplots(1, 2, figsize=figsize)
            
            # -------------------- PEARSON --------------------
            mask_pearson = np.triu(np.ones_like(matrix_pearson, dtype=bool), k=1)
            
            sns.heatmap(
                matrix_pearson,
                annot=True,
                fmt=".2f",
                cmap='RdBu_r',  # Vermelho = positivo, Azul = negativo
                center=0,
                vmin=-1,
                vmax=1,
                square=True,
                linewidths=0.5,
                cbar_kws={
                    "shrink": 0.8,
                    "label": "Correlação"
                },
                ax=axes[0],
                xticklabels=labels_curtos,
                yticklabels=labels_curtos,
                annot_kws={"size": 8}
            )
            
            axes[0].set_title(
                "Matriz de Pearson\n(Correlação Linear)",
                fontsize=14,
                fontweight='bold',
                pad=15
            )
            axes[0].set_xticklabels(
                axes[0].get_xticklabels(),
                rotation=45,
                ha='right',
                fontsize=9
            )
            axes[0].set_yticklabels(
                axes[0].get_yticklabels(),
                rotation=0,
                fontsize=9
            )
            
            # -------------------- SPEARMAN --------------------
            mask_spearman = np.triu(np.ones_like(matrix_spearman, dtype=bool), k=1)
            
            sns.heatmap(
                matrix_spearman,
                annot=True,
                fmt=".2f",
                cmap='PiYG',  # Rosa/Verde para Spearman
                center=0,
                vmin=-1,
                vmax=1,
                square=True,
                linewidths=0.5,
                cbar_kws={
                    "shrink": 0.8,
                    "label": "Correlação"
                },
                ax=axes[1],
                xticklabels=labels_curtos,
                yticklabels=labels_curtos,
                annot_kws={"size": 8}
            )
            
            axes[1].set_title(
                "Matriz de Spearman\n(Correlação Monotônica)",
                fontsize=14,
                fontweight='bold',
                pad=15
            )
            axes[1].set_xticklabels(
                axes[1].get_xticklabels(),
                rotation=45,
                ha='right',
                fontsize=9
            )
            axes[1].set_yticklabels(
                axes[1].get_yticklabels(),
                rotation=0,
                fontsize=9
            )
            
            plt.tight_layout()
            plt.show()
            plt.close()
            
            # ============================================================
            # VISUALIZAÇÃO ALTERNATIVA: Apenas Triângulo Superior
            # ============================================================
            
            print("\n📊 Gerando visualização alternativa (triângulo superior)...\n")
            
            fig, axes = plt.subplots(1, 2, figsize=figsize)
            
            # Máscara para triângulo superior
            mask = np.triu(np.ones_like(matrix_pearson, dtype=bool))
            
            # Pearson (triângulo inferior)
            sns.heatmap(
                matrix_pearson,
                mask=mask,
                annot=True,
                fmt=".2f",
                cmap='RdBu_r',
                center=0,
                vmin=-1,
                vmax=1,
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8},
                ax=axes[0],
                xticklabels=labels_curtos,
                yticklabels=labels_curtos,
                annot_kws={"size": 8}
            )
            
            axes[0].set_title(
                "Pearson (Triângulo Inferior)",
                fontsize=14,
                fontweight='bold',
                pad=15
            )
            axes[0].set_xticklabels(
                axes[0].get_xticklabels(),
                rotation=45,
                ha='right',
                fontsize=9
            )
            axes[0].set_yticklabels(
                axes[0].get_yticklabels(),
                rotation=0,
                fontsize=9
            )
            
            # Spearman (triângulo inferior)
            sns.heatmap(
                matrix_spearman,
                mask=mask,
                annot=True,
                fmt=".2f",
                cmap='PiYG',
                center=0,
                vmin=-1,
                vmax=1,
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8},
                ax=axes[1],
                xticklabels=labels_curtos,
                yticklabels=labels_curtos,
                annot_kws={"size": 8}
            )
            
            axes[1].set_title(
                "Spearman (Triângulo Inferior)",
                fontsize=14,
                fontweight='bold',
                pad=15
            )
            axes[1].set_xticklabels(
                axes[1].get_xticklabels(),
                rotation=45,
                ha='right',
                fontsize=9
            )
            axes[1].set_yticklabels(
                axes[1].get_yticklabels(),
                rotation=0,
                fontsize=9
            )
            
            plt.tight_layout()
            plt.show()
            plt.close()
        
        return matrix_pearson, matrix_spearman

In [ ]:
def plot_correlation_clusters(matrix_pearson, threshold=0.85, figsize=(16, 12), max_pairs=30):
    """
    Visualiza apenas pares com alta correlação (|r| >= threshold).
    Versão otimizada para evitar sobreposição de labels.
    
    Parâmetros:
    -----------
    matrix_pearson : pd.DataFrame - Matriz de correlação
    threshold : float - Threshold mínimo de correlação
    figsize : tuple - Tamanho da figura
    max_pairs : int - Número máximo de pares a exibir
    """
    
    # Extrair pares com alta correlação
    pares_altos = []
    n = len(matrix_pearson)
    
    for i in range(n):
        for j in range(i+1, n):
            corr_val = matrix_pearson.iloc[i, j]
            if abs(corr_val) >= threshold:
                pares_altos.append({
                    'feature_1': matrix_pearson.index[i],
                    'feature_2': matrix_pearson.columns[j],
                    'correlacao': corr_val
                })
    
    if not pares_altos:
        print(f"✅ Nenhum par com |correlação| >= {threshold}")
        return
    
    # Criar DataFrame e ordenar
    df_pares = pd.DataFrame(pares_altos).sort_values('correlacao', key=abs, ascending=False)
    
    # Limitar número de pares se necessário
    if len(df_pares) > max_pairs:
        print(f"⚠️  Limitando visualização aos top {max_pairs} pares (de {len(df_pares)} encontrados)")
        df_pares = df_pares.head(max_pairs)
    
    print(f"\n🔗 Encontrados {len(pares_altos)} pares com |correlação| >= {threshold}\n")
    print(df_pares.to_string(index=False))
    
    # ========================================================================
    # VISUALIZAÇÃO MELHORADA
    # ========================================================================
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Criar labels mais limpos
    def truncar_nome(nome, max_len=35):
        """Trunca nome mantendo início e fim"""
        if len(nome) <= max_len:
            return nome
        else:
            # Manter início e fim
            metade = (max_len - 3) // 2
            return f"{nome[:metade]}...{nome[-metade:]}"
    
    # Labels formatados (um por linha, sem "vs")
    labels = []
    for _, row in df_pares.iterrows():
        feat1 = truncar_nome(row['feature_1'], max_len=40)
        feat2 = truncar_nome(row['feature_2'], max_len=40)
        labels.append(f"{feat1}\n  ⟷  {feat2}")
    
    # Cores por intensidade
    def get_color(corr):
        abs_corr = abs(corr)
        if abs_corr >= 0.99:
            return '#8B0000'  # Vermelho escuro (perfeita)
        elif abs_corr >= 0.95:
            return '#DC143C'  # Vermelho (crítica)
        elif abs_corr >= 0.90:
            return '#FF8C00'  # Laranja (muito alta)
        else:
            return '#32CD32'  # Verde (alta)
    
    colors = [get_color(c) for c in df_pares['correlacao']]
    
    # Criar barras horizontais
    y_pos = np.arange(len(df_pares))
    bars = ax.barh(y_pos, df_pares['correlacao'], color=colors, alpha=0.85, height=0.7)
    
    # Configurar eixos
    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels, fontsize=9, family='monospace')
    ax.set_xlabel('Correlação de Pearson', fontsize=13, fontweight='bold')
    ax.set_xlim([-1.05, 1.15])  # Espaço extra para valores
    
    # Título
    ax.set_title(
        f'Pares com Alta Correlação (|r| ≥ {threshold})\n'
        f'Total: {len(pares_altos)} pares | Exibindo: {len(df_pares)} pares',
        fontsize=15,
        fontweight='bold',
        pad=20
    )
    
    # Linhas de referência
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1.2, alpha=0.7)
    ax.axvline(x=threshold, color='red', linestyle='--', linewidth=1.5, alpha=0.4, 
               label=f'Threshold = ±{threshold}')
    ax.axvline(x=-threshold, color='red', linestyle='--', linewidth=1.5, alpha=0.4)
    ax.axvline(x=0.95, color='orange', linestyle=':', linewidth=1.2, alpha=0.4, 
               label='Crítico = ±0.95')
    ax.axvline(x=-0.95, color='orange', linestyle=':', linewidth=1.2, alpha=0.4)
    
    # Grid
    ax.grid(axis='x', alpha=0.3, linestyle='--', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Adicionar valores nas barras
    for i, (bar, val) in enumerate(zip(bars, df_pares['correlacao'])):
        # Posição do texto
        x_pos = val + 0.03 if val > 0 else val - 0.03
        ha = 'left' if val > 0 else 'right'
        
        # Cor do texto (branco se barra escura)
        text_color = 'white' if abs(val) >= 0.95 else 'black'
        
        # Texto dentro da barra se correlação alta
        if abs(val) >= 0.90:
            x_pos = val - 0.03 if val > 0 else val + 0.03
            ha = 'right' if val > 0 else 'left'
            text_color = 'white'
        
        ax.text(
            x_pos,
            i,
            f'{val:.3f}',
            va='center',
            ha=ha,
            fontsize=10,
            fontweight='bold',
            color=text_color
        )
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#8B0000', label='Perfeita (≥ 0.99)'),
        Patch(facecolor='#DC143C', label='Crítica (0.95 - 0.99)'),
        Patch(facecolor='#FF8C00', label='Muito Alta (0.90 - 0.95)'),
        Patch(facecolor='#32CD32', label='Alta (0.85 - 0.90)'),
    ]
    ax.legend(
        handles=legend_elements,
        loc='lower right',
        fontsize=10,
        framealpha=0.9,
        title='Intensidade'
    )
    
    plt.tight_layout()
    plt.show()
    plt.close()
    
    # ========================================================================
    # TABELA RESUMIDA POR CATEGORIA
    # ========================================================================
    
    print("\n" + "="*100)
    print("📊 RESUMO POR CATEGORIA DE CORRELAÇÃO")
    print("="*100)
    
    categorias = [
        ('🔴 PERFEITA (≥ 0.99)', 0.99, 1.01),
        ('🟠 CRÍTICA (0.95 - 0.99)', 0.95, 0.99),
        ('🟡 MUITO ALTA (0.90 - 0.95)', 0.90, 0.95),
        ('🟢 ALTA (0.85 - 0.90)', 0.85, 0.90),
    ]
    
    for titulo, min_val, max_val in categorias:
        subset = df_pares[
            (df_pares['correlacao'].abs() >= min_val) & 
            (df_pares['correlacao'].abs() < max_val)
        ]
        
        if len(subset) > 0:
            print(f"\n{titulo} ({len(subset)} pares)")
            print("-"*100)
            
            for idx, row in subset.iterrows():
                feat1 = row['feature_1'][:45]
                feat2 = row['feature_2'][:45]
                corr = row['correlacao']
                
                print(f"  • {feat1:47s} ⟷  {feat2:47s}  │  r = {corr:+.4f}")
    
    print("\n" + "="*100)

In [ ]:
def calcular_cramer_v(df, col1, col2):
    # Verificação de segurança: não rodar em colunas com muitos valores únicos (numéricas)
    # Se a coluna tiver mais de 100 valores distintos, provavelmente não é uma categoria útil
    distinct_count = df.select(col1).distinct().count()
    if distinct_count > 100:
        print(f"Pulei {col1}: muitos valores únicos ({distinct_count}). Verifique se é quantitativa.")
        return np.nan

    # Indexação
    idx1, idx2 = col1 + "_idx", col2 + "_idx"
    df_indexed = StringIndexer(inputCol=col1, outputCol=idx1, handleInvalid="skip").fit(df).transform(df)
    df_indexed = StringIndexer(inputCol=col2, outputCol=idx2, handleInvalid="skip").fit(df_indexed).transform(df_indexed)

    # Assembler para o Teste
    assembler = VectorAssembler(inputCols=[idx1], outputCol="features")
    test_data = assembler.transform(df_indexed).select("features", idx2)

    # Chi-Square Test
    res = ChiSquareTest.test(test_data, "features", idx2).head()
    
    # --- CORREÇÃO DO ACESSO AOS ATRIBUTOS ---
    chi2 = float(res.statistics[0]) # Estatística Chi2 correta
    n = test_data.count()
    
    # Resgate do número de categorias pelos metadados ou contagem
    r = df_indexed.select(idx1).distinct().count()
    k = df_indexed.select(idx2).distinct().count()

    if n == 0 or min(r-1, k-1) <= 0: return 0.0
    
    v = np.sqrt(chi2 / (n * min(r-1, k-1)))
    return v

In [ ]:
def matriz_cramer_v_spark_optimized(df, colunas_categoricas, figsize=(20, 16)):
    """
    Constrói uma matriz de correlação Cramer's V para variáveis categóricas.
    Otimizada para visualização de até 20+ variáveis.
    
    Parameters:
    -----------
    df : pyspark.sql.DataFrame
        DataFrame com as variáveis categóricas
    colunas_categoricas : list
        Lista de nomes das colunas categóricas
    figsize : tuple
        Tamanho da figura (width, height)
    
    Returns:
    --------
    pd.DataFrame : Matriz de correlação Cramer's V
    """
    
    n_cols = len(colunas_categoricas)
    matrix = np.ones((n_cols, n_cols))
    
    print(f"🔄 Iniciando indexação de {n_cols} colunas categóricas...")
    
    # ═══════════════════════════════════════════════════════════
    # 1. INDEXAÇÃO EM BATCH (Performance)
    # ═══════════════════════════════════════════════════════════
    indexers = [
        StringIndexer(
            inputCol=c, 
            outputCol=c+"_idx", 
            handleInvalid="keep"  # Mantém categorias novas como "unknown"
        ) 
        for c in colunas_categoricas
    ]
    
    df_indexed = df
    models = []
    for idx, indexer in enumerate(indexers):
        model = indexer.fit(df_indexed)
        df_indexed = model.transform(df_indexed)
        models.append(model)
    
    print(f"✅ Indexação concluída!\n")
    
    # ═══════════════════════════════════════════════════════════
    # 2. CÁLCULO DO CRAMER'S V (Triângulo Superior)
    # ═══════════════════════════════════════════════════════════
    total_pairs = (n_cols * (n_cols - 1)) // 2
    current_pair = 0
    
    for i in range(n_cols):
        for j in range(i + 1, n_cols):
            current_pair += 1
            col1 = colunas_categoricas[i]
            col2 = colunas_categoricas[j]
            col1_idx = col1 + "_idx"
            col2_idx = col2 + "_idx"
            
            print(f"Calculando: {col1} ↔ {col2}")
            
            # Preparar dados para ChiSquareTest
            assembler = VectorAssembler(inputCols=[col1_idx], outputCol="features")
            test_data = assembler.transform(df_indexed).select("features", col2_idx)
            
            # Executar teste Qui-Quadrado
            try:
                res = ChiSquareTest.test(test_data, "features", col2_idx).head()
                chi2 = float(res.statistics[0])
                n = test_data.count()
                
                # Número de categorias (r e k)
                r = len(models[i].labels)
                k = len(models[j].labels)
                
                # Cálculo do V de Cramer
                if n > 0 and min(r-1, k-1) > 0:
                    v = np.sqrt(chi2 / (n * min(r-1, k-1)))
                else:
                    v = 0.0
                    
            except Exception as e:
                print(f"  ⚠️ Erro ao calcular {col1} vs {col2}: {e}")
                v = 0.0
                
            # Preencher matriz (simétrica)
            matrix[i, j] = v
            matrix[j, i] = v
    
    print(f"\n✅ Cálculo concluído!\n")
    
    # ═══════════════════════════════════════════════════════════
    # 3. CRIAR DATAFRAME E VISUALIZAÇÃO OTIMIZADA
    # ═══════════════════════════════════════════════════════════
    cramer_df = pd.DataFrame(
        matrix, 
        index=colunas_categoricas, 
        columns=colunas_categoricas
    )
    
    # Plotar Heatmap Otimizado
    fig, ax = plt.subplots(figsize=figsize)
    
    # Máscara para o triângulo superior (opcional, deixa mais limpo)
    mask = np.triu(np.ones_like(cramer_df, dtype=bool), k=1)
    
    sns.heatmap(
        cramer_df, 
        annot=True,           # Mostrar valores
        fmt='.2f',            # 2 casas decimais
        cmap='RdYlGn_r',      # Colormap: Vermelho (alta correlação) -> Verde (baixa)
        vmin=0, 
        vmax=1,
        center=0.5,           # Centro da escala
        square=True,          # Células quadradas
        linewidths=0.5,       # Linhas entre células
        cbar_kws={
            "shrink": 0.8,
            "label": "Cramer's V"
        },
        mask=mask,            # Oculta triângulo superior (opcional)
        ax=ax
    )
    
    # Ajustes de visualização
    plt.title(
        "Matriz de Associação: Cramer's V (Variáveis Categóricas)", 
        fontsize=16, 
        fontweight='bold',
        pad=20
    )
    plt.xlabel("", fontsize=12)
    plt.ylabel("", fontsize=12)
    
    # Rotacionar labels para melhor legibilidade
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    return cramer_df

In [ ]:
def matriz_cramer_v_spark(df, colunas_categoricas):
    """
    Constrói uma matriz de correlação Cramer's V para uma lista de colunas.
    """
    n_cols = len(colunas_categoricas)
    # Inicializa a matriz com 1s na diagonal (correlação de uma var com ela mesma)
    matrix = np.ones((n_cols, n_cols))
    
    print(f"Iniciando indexação de {n_cols} colunas...")
    
    # 1. Indexar todas as colunas de uma vez para ganhar performance
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="skip") for c in colunas_categoricas]
    
    # Aplicamos a indexação em cadeia
    df_indexed = df
    models = []
    for indexer in indexers:
        model = indexer.fit(df_indexed)
        df_indexed = model.transform(df_indexed)
        models.append(model)
    
    # 2. Iterar sobre os pares (apenas o triângulo superior para economizar processamento)
    for i in range(n_cols):
        for j in range(i + 1, n_cols):
            col1 = colunas_categoricas[i]
            col2 = colunas_categoricas[j]
            col1_idx = col1 + "_idx"
            col2_idx = col2 + "_idx"
            
            print(f"Calculando Cramer's V: {col1} vs {col2}...")
            
            # Preparar dados para o ChiSquareTest
            assembler = VectorAssembler(inputCols=[col1_idx], outputCol="features")
            test_data = assembler.transform(df_indexed).select("features", col2_idx)
            
            # Executar teste
            res = ChiSquareTest.test(test_data, "features", col2_idx).head()
            chi2 = float(res.statistics[0])
            n = test_data.count()
            
            # Número de categorias de cada coluna (r e k) extraídos dos modelos indexadores
            r = len(models[i].labels)
            k = len(models[j].labels)
            
            # Cálculo do V de Cramer
            if n > 0 and min(r-1, k-1) > 0:
                v = np.sqrt(chi2 / (n * min(r-1, k-1)))
            else:
                v = 0.0
                
            # Preencher a matriz de forma simétrica
            matrix[i, j] = v
            matrix[j, i] = v
            
    # Criar DataFrame Pandas para facilitar a visualização
    cramer_df = pd.DataFrame(matrix, index=colunas_categoricas, columns=colunas_categoricas)
    
    # Plotar o Heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cramer_df, annot=True, cmap='YlGnBu', fmt='.2f', vmin=0, vmax=1)
    plt.title("Matriz de Associação: Cramer's V (Variáveis Categóricas)")
    plt.show()
    
    return cramer_df

In [ ]:
def identificar_outliers(df, coluna, negativo=True, minimo_definido=0):
    # Calcula os quantis de forma aproximada para performance
    quantis = df.approxQuantile(coluna, [0.25, 0.75], 0.05)
    q1, q3 = quantis[0], quantis[1]
    iqr = q3 - q1

    limite_superior = q3 + 1.5 * iqr
    
    # Se negativo for True, permite valores negativos no limite inferior --> dependendo da variável
    if negativo:
        limite_inferior = q1 - 1.5 * iqr
    else:
        limite_inferior = max(0, (q1 - 1.5 * iqr))

    if minimo_definido != 0:
        limite_inferior = minimo_definido
    
    outliers = df.filter((F.col(coluna) < limite_inferior) | (F.col(coluna) > limite_superior))
    print(f"Variável {coluna}:")
    print(f"Limites: [{limite_inferior}, {limite_superior}]")
    print(f"Total de outliers (#): {outliers.count()}")
    print(f"Total de outliers (%): {(outliers.count() / df.count()) * 100:.2f}%")

In [ ]:
def plot_tendencia_temporal(df, col_valor, aggregation="mean", col_safra="safra", categories=None):
    """
    df: Spark DataFrame
    col_valor: Variável numérica para calcular a média
    col_safra: Coluna de tempo (ex: 'safra')
    categories: Nome da coluna categórica para quebrar as linhas (ex: 'gender') ou None
    """
    # Definir a lista de colunas para o agrupamento
    group_cols = [col_safra]
    if categories:
        group_cols.append(categories)
    
    # Agregação no Spark por safra (e categoria, se houver) e calculamos a média. Transformar em Pandas para plotar
    if aggregation == "mean":
        stats_safra = (df.groupBy(group_cols)
                        .agg(F.mean(col_valor).alias(f"{aggregation}"))
                        .orderBy(col_safra)
                        .toPandas())
    elif aggregation == "sum":
        stats_safra = (df.groupBy(group_cols)
                        .agg(F.sum(col_valor).alias(f"{aggregation}"))
                        .orderBy(col_safra)
                        .toPandas())
    elif aggregation == "max":
        stats_safra = (df.groupBy(group_cols)
                        .agg(F.max(col_valor).alias(f"{aggregation}"))
                        .orderBy(col_safra)
                        .toPandas())
    elif aggregation == "min":
        stats_safra = (df.groupBy(group_cols)
                        .agg(F.min(col_valor).alias(f"{aggregation}"))
                        .orderBy(col_safra)
                        .toPandas())
    elif aggregation == "med":
        stats_safra = (df.groupBy(group_cols)
                        .agg(F.expr("percentile_approx({}, 0.5)".format(col_valor)).alias(f"{aggregation}"))
                        .orderBy(col_safra)
                        .toPandas())
    
    # Tratar a safra como categoria (texto) 
    stats_safra[col_safra] = stats_safra[col_safra].astype(str)
    
    # 4. Configuração do Plot
    plt.figure(figsize=(14, 6))
    
    # Se 'categories' for informada, o Seaborn cria uma linha para cada categoria usando 'hue'
    sns.lineplot(
        data=stats_safra, 
        x=col_safra, 
        y=f"{aggregation}", 
        hue=categories if categories else None, 
        marker='o', 
        sort=False # Garante que a ordem das safras do Spark seja mantida
    )
    
    # Ajustes finos de visualização
    titulo = f"{aggregation} de {col_valor} por {col_safra}"
    if categories:
        titulo += f" (Quebrado por {categories})"
        plt.legend(title=categories, bbox_to_anchor=(1.02, 1), loc='upper left')
        
    plt.title(titulo)
    plt.xticks(rotation=45)
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
def integridade_entre_bases(df_base, df_comparacao, nome_relacao, chave=["msno", "safra"]):
    """
    Rastreia a perda de dados entre tabelas (ex: membros vs logs).
    Essencial para justificar a volumetria final do modelo.
    """
    total_base = df_base.select(chave).distinct().count()
    com_match = df_base.join(df_comparacao, chave, "inner").select(chave).distinct().count()
    perda = ((total_base - com_match) / total_base) * 100
    
    print(f"--- Integridade: {nome_relacao} ---")
    print(f"Registros na base: {total_base} | Registros correspondentes: {com_match}")
    print(f"Taxa de Perda: {perda:.2f}%\n")

In [ ]:
def plot_boxplot(df, coluna_categorica, coluna_alvo, agrupar_por_safra=False, table=False):
    """
    Plota boxplots utilizando estatísticas calculadas no Spark, incluindo a média.
    """
    
    # 1. Definimos as colunas de agrupamento
    cols_base = ['safra'] if agrupar_por_safra else []

    for col_cat in coluna_categorica:
        print(f"Processando estatísticas para: {col_cat}...")
        
        # 2. Calculamos os quartis E a média no Spark
        df_stats = (df
                    .groupBy(cols_base + [col_cat])
                    .agg(
                        F.percentile_approx(coluna_alvo, [0.0, 0.25, 0.5, 0.75, 1.0], 10000).alias("stats"),
                        F.mean(coluna_alvo).alias("avg") # <-- Adicionado média
                    )
                    .select(*cols_base, col_cat,
                            F.col("stats")[0].alias("min"),
                            F.col("stats")[1].alias("q1"),
                            F.col("stats")[2].alias("med"),
                            F.col("stats")[3].alias("q3"),
                            F.col("stats")[4].alias("max"),
                            F.col("avg").alias("mean")) # <-- Selecionando a média
                    .toPandas())

        # 3. Lógica de Plotagem
        if agrupar_por_safra:
            safras = sorted(df_stats['safra'].unique())
            for s in safras:
                df_safra = df_stats[df_stats['safra'] == s]
                _desenhar_boxplot_estatistico(df_safra, col_cat, coluna_alvo, f"Safra {s}", show_table=table)
        else:
            _desenhar_boxplot_estatistico(df_stats, col_cat, coluna_alvo, "Visão Consolidada", show_table=table)

def _desenhar_boxplot_estatistico(df_plot, col_cat, col_alvo, subtitulo, show_table=False):
    stats_list = []
    df_plot = df_plot.sort_values(by=col_cat)

    for _, row in df_plot.iterrows():
        stats_list.append({
            "label": str(row[col_cat]),
            "whislo": row["min"], 
            "q1": row["q1"],
            "med": row["med"],
            "q3": row["q3"],
            "whishi": row["max"],
            "mean": row["mean"] # <-- Matplotlib usa essa chave para desenhar a média
        })

    if not stats_list:
        return

    # Renderização do Gráfico
    plt.figure(figsize=(12, 6))
    ax = plt.gca()
    # showmeans=True ativa a exibição do marcador da média
    ax.bxp(stats_list, showfliers=False, showmeans=True, 
           meanprops={"marker":"o", "markerfacecolor":"white", "markeredgecolor":"black", "markersize":"6"}) 
    
    plt.title(f"Boxplot: {col_alvo} por {col_cat} ({subtitulo})")
    plt.ylabel(col_alvo)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Renderização da Tabela
    if show_table:
        print(f"\n--- Estatísticas: {col_cat} ({subtitulo}) ---")
        # Adicionado "mean" na visualização da tabela
        cols_tabela = [col_cat, "min", "q1", "med", "mean", "q3", "max"]
        df_resumo = df_plot[cols_tabela].copy()
        
        try:
            from IPython.display import display
            display(df_resumo)
        except ImportError:
            print(df_resumo.to_string(index=False))
        print("\n" + "="*50 + "\n")

## 3.2. Tratamento de dados

### 3.2.1. Agrupamento

In [ ]:
def agregar_logs(df_logs):
    """
    Agrega a tabela de logs no nível (msno, safra).
    
    Premissa: A base de logs já vem agregada por mês, mas esta função
    garante robustez caso surjam duplicatas em produção.
    
    Regra de agregação:
    - Variáveis numéricas de uso: SUM (acumulação de comportamento)
    
    Args:
        df_logs: DataFrame Spark com logs brutos
        
    Returns:
        DataFrame agregado no nível (msno, safra)
    """
    
    # Colunas esperadas (contrato de dados)
    expected_cols = ["msno", "safra", "num_25", "num_50", "num_75", "num_985", "num_100", "num_unq", "total_secs"]
    
    # Selecionar apenas colunas esperadas (proteção contra novas features)
    df_logs = df_logs.select(*expected_cols)
    
    # Verificar duplicatas
    duplicates = df_logs.groupBy("msno", "safra").count().filter("count > 1")
    
    if duplicates.count() > 0:
        print(f"⚠️ LOGS: {duplicates.count()} duplicatas detectadas. Aplicando agregação...")
        
        df_logs_agg = (
            df_logs
            .groupBy("msno", "safra")
            .agg(
                # Variáveis de uso: acumulação (SUM)
                F.sum("num_25").alias("num_25"),
                F.sum("num_50").alias("num_50"),
                F.sum("num_75").alias("num_75"),
                F.sum("num_985").alias("num_985"),
                F.sum("num_100").alias("num_100"),
                F.sum("num_unq").alias("num_unq"),
                F.sum("total_secs").alias("total_secs")))
        
        print("✅ LOGS: Agregação concluída.")
        return df_logs_agg
    
    else:
        print("✅ LOGS: Base já está no nível (msno, safra). Nenhuma agregação necessária.")
        return df_logs

In [ ]:
def agregar_transactions(df_transactions):
    """
    Agrega a tabela de transações no nível (msno, safra).
    
    Premissa: Podem existir múltiplas transações por cliente no mesmo mês
    (upgrades, downgrades, reprocessamentos).
    
    Regras de agregação:
    - Flags: MAX (ocorreu ao menos uma vez?)
    - Valores monetários: SUM (acumulação financeira)
    - Atributos de plano e categóricas: LAST (último estado observado)
    - Datas: MAX para transaction_date, LAST para membership_expire_date
    
    Args:
        df_transactions: DataFrame Spark com transações brutas
        
    Returns:
        DataFrame agregado no nível (msno, safra)
    """
    
    # Colunas esperadas (contrato de dados)
    expected_cols = ["msno", "safra", "payment_method_id", "payment_plan_days", "plan_list_price",
        "actual_amount_paid", "is_auto_renew", "transaction_date", "membership_expire_date", "is_cancel"]
    
    # Selecionar apenas colunas esperadas (proteção contra novas features)
    df_transactions = df_transactions.select(*expected_cols)
    
    # Verificar duplicatas
    duplicates = df_transactions.groupBy("msno", "safra").count().filter("count > 1")
    
    if duplicates.count() > 0:
        print(f"⚠️ TRANSACTIONS: {duplicates.count()} duplicatas detectadas. Aplicando agregação...")
        
        # Criar ranking por data (última transação = 1)
        w_order = Window.partitionBy("msno", "safra").orderBy(F.col("transaction_date").desc())
        df_transactions = df_transactions.withColumn("_rank", F.row_number().over(w_order))
        
        df_tx_agg = (
            df_transactions
            .groupBy("msno", "safra")
            .agg(
                # Flags: MAX (ocorrência)
                F.max("is_auto_renew").alias("is_auto_renew"),
                F.max("is_cancel").alias("is_cancel"),
                
                # Valores monetários: SUM (acumulação)
                F.sum("actual_amount_paid").alias("actual_amount_paid"),
                
                # Atributos de plano: LAST (último estado)
                F.max(F.when(F.col("_rank") == 1, F.col("payment_method_id"))).alias("payment_method_id"),
                F.max(F.when(F.col("_rank") == 1, F.col("payment_plan_days"))).alias("payment_plan_days"),
                F.max(F.when(F.col("_rank") == 1, F.col("plan_list_price"))).alias("plan_list_price"),
                
                # Datas
                F.max("transaction_date").alias("transaction_date"),
                F.max(F.when(F.col("_rank") == 1, F.col("membership_expire_date"))).alias("membership_expire_date")
            ))
        
        print("✅ TRANSACTIONS: Agregação concluída.")
        return df_tx_agg
    
    else:
        print("✅ TRANSACTIONS: Base já está no nível (msno, safra). Nenhuma agregação necessária.")
        return df_transactions

In [ ]:
def agregar_members(df_members):
    """
    Agrega a tabela de membros no nível (msno, safra).
    
    Premissa: Atributos de membros são majoritariamente estáticos, mas podem
    existir múltiplas linhas por cliente em casos de mudança de cidade, 
    reativação, ou inconsistências de dados.
    
    Regras de agregação:
    - Atributos demográficos: LAST (último estado observado)
    - Flag de status: MAX (esteve ativo ao menos uma vez?)
    - Data de registro: MIN (primeira ocorrência)
    
    Args:
        df_members: DataFrame Spark com dados de membros
        
    Returns:
        DataFrame agregado no nível (msno, safra)
    """
    
    # Colunas esperadas (contrato de dados)
    expected_cols = ["msno", "safra","registration_init_time", "city", "bd", "gender", "registered_via", "is_ativo"]
    
    # Selecionar apenas colunas esperadas (proteção contra novas features)
    df_members = df_members.select(*expected_cols)
    
    # Converter safra para integer (está como string no schema)
    df_members = df_members.withColumn("safra", F.col("safra").cast("integer"))
    
    # Verificar duplicatas
    duplicates = df_members.groupBy("msno", "safra").count().filter("count > 1")
    
    if duplicates.count() > 0:
        print(f"⚠️ MEMBERS: {duplicates.count()} duplicatas detectadas. Aplicando agregação...")
        
        # Criar ranking por data de registro (mais recente = 1)
        # Se não houver coluna de atualização, usamos registration_init_time como proxy
        w_order = Window.partitionBy("msno", "safra").orderBy(F.col("registration_init_time").desc())
        df_members = df_members.withColumn("_rank", F.row_number().over(w_order))
        
        df_members_agg = (
            df_members
            .groupBy("msno", "safra")
            .agg(
                # Data de registro: MIN (primeira ocorrência)
                F.min("registration_init_time").alias("registration_init_time"),
                
                # Atributos demográficos: LAST (último estado)
                F.max(F.when(F.col("_rank") == 1, F.col("city"))).alias("city"),
                F.max(F.when(F.col("_rank") == 1, F.col("bd"))).alias("bd"),
                F.max(F.when(F.col("_rank") == 1, F.col("gender"))).alias("gender"),
                F.max(F.when(F.col("_rank") == 1, F.col("registered_via"))).alias("registered_via"),
                
                # Flag de status: MAX (esteve ativo?)
                F.max("is_ativo").alias("is_ativo")))
        
        print("✅ MEMBERS: Agregação concluída.")
        return df_members_agg
    
    else:
        print("✅ MEMBERS: Base já está no nível (msno, safra). Nenhuma agregação necessária.")
        return df_members

### 3.2.2. Outliers e Categorização Estatística

In [ ]:
def aplicar_winsorizacao(df, colunas, p_inf=0.01, p_sup=0.99):
    for col in colunas:
        # Calcula os limites baseados em percentis
        limites = df.approxQuantile(col, [p_inf, p_sup], 0.001) # 0.001 é a precisão/erro aceitável
        inf, sup = limites[0], limites[1]
        
        print(f"Coluna {col}: Limite Inferior={inf}, Limite Superior={sup}")
        
        # Aplica o Capping
        df = df.withColumn(f"{col}_win", 
            F.when(F.col(col) < inf, inf)
             .when(F.col(col) > sup, sup)
             .otherwise(F.col(col))
        )
    return df

In [ ]:
def segment_by_percentile(df, col_to_group, table_name, num_buckets=4):
    """
    Categoriza uma variável dinamicamente com base em percentis calculados no Spark.
    
    Parâmetros:
    - df: DataFrame do Spark.
    - col_to_group: Nome da coluna numérica a ser segmentada (ex: 'total_plays').
    - table_name: Nome da tabela de origem para compor a flag (ex: 'logs').
    - num_buckets: Número de divisões desejadas. 
                   4 para Quartis (25%, 50%, 75%, 100%)
                   10 para Decisões (10%, 20%, ..., 100%)
    
    Variáveis esperadas no DataFrame:
    - f"flag_has_{table_name}": Coluna binária (0/1) que indica presença de dados.
    - {col_to_group}: A variável numérica que será processada.
    """
    
    # 1. Definimos a coluna de flag
    flag_col = f"flag_has_{table_name}"
    
    # 2. Calculo dinamico dos percentis
    # Criamos uma lista de probabilidades: ex para 4 buckets -> [0.25, 0.50, 0.75, 1.0]
    probabilities = [i / num_buckets for i in range(1, num_buckets + 1)]
    
    # Coletamos os valores de corte (stats)
    # Nota: Filtramos apenas onde a flag é 1 para não enviesar os percentis com zeros/nulos
    cut_off_points = (df
                      .filter(F.col(flag_col).isin(1))
                      .agg(F.percentile_approx(col_to_group, probabilities).alias("pts"))
                      .collect()[0]["pts"])
    
    # 3. A lógica do .when dinamico, comecando com a condicao base para desconhecidos/inativos
    case_expression = F.when(F.col(flag_col).isin(0), "unknown")
    
    # Iteramos sobre os pontos de corte para criar as faixas
    # Ex: para Quartis, teremos Tier 1 (até 25%), Tier 2 (até 50%), etc.
    for i, point in enumerate(cut_off_points):
        tier_label = f"tier_{i+1}"
        # A primeira faixa pega do menor valor até o primeiro ponto
        if i == 0:
            case_expression = case_expression.when(F.col(col_to_group) <= point, tier_label)
        else:
            # As demais pegam entre o ponto anterior e o atual
            case_expression = case_expression.when(
                (F.col(col_to_group) > cut_off_points[i-1]) & (F.col(col_to_group) <= point), 
                tier_label)
            
    # Caso escape de alguma lógica (fallback)
    case_expression = case_expression.otherwise(f"tier_{num_buckets}")

    # Retornamos o dataframe com a nova coluna
    return df.withColumn(f"{col_to_group}_group", case_expression)

### 3.2.3. Tendência - Janela temporal (3 e 6 meses)

In [ ]:
def features_de_tendencia_num(df, variables, id_col="msno", time_col="safra", windows=[3, 6], keep_intermediate=False):
    original_cols = df.columns
    generated_cols = []

    for var in variables:
        # manter raw
        raw_col = f"{var}_raw"
        df = df.withColumn(raw_col, F.col(var))
        generated_cols.append(raw_col)

        for n in windows:
            w = Window.partitionBy(id_col).orderBy(time_col)

            # lags
            lags = []
            for i in range(n):
                lag_col = f"{var}_lag_{i}"
                df = df.withColumn(lag_col, F.lag(var, i).over(w))
                lags.append(lag_col)

            # contadores
            cnt_null = f"{var}_cnt_null_{n}"
            cnt_zero = f"{var}_cnt_zero_{n}"

            df = (df
                .withColumn(cnt_null, sum(F.col(c).isNull().cast("int") for c in lags))
                .withColumn(cnt_zero, sum((F.col(c) == 0).cast("int") for c in lags)))

            ref = f"{var}_lag_0"
            arr = f"array({','.join(lags)})"

            # métricas raw
            mean_raw = f"{var}_mean_{n}_raw"
            min_raw = f"{var}_min_{n}_raw"
            max_raw = f"{var}_max_{n}_raw"

            df = (df
                .withColumn(
                    mean_raw,
                    F.expr(f"""
                        aggregate(filter({arr}, x -> x is not null), 0D, (acc, x) -> acc + x) / size(filter({arr}, x -> x is not null))
                        """))
                .withColumn(min_raw, F.expr(f"array_min(filter({arr}, x -> x is not null))"))
                .withColumn(max_raw, F.expr(f"array_max(filter({arr}, x -> x is not null))")))

            # métricas finais (com sentinela)
            for metric, raw_metric in zip(
                ["mean", "min", "max"],
                [mean_raw, min_raw, max_raw]
            ):
                final_col = f"{var}_{metric}_{n}"
                generated_cols.append(final_col)

                df = df.withColumn(final_col,
                    F.when(F.col(cnt_null) == n, F.lit(-99998))
                     .when(F.col(cnt_zero) == n, F.lit(-99999))
                     .when((F.col(ref) == 0) & (F.col(cnt_null) < n), F.lit(-99995))
                     .when((F.col(ref).isNotNull()) & (F.col(cnt_null) == n - 1), F.lit(-99996))
                     .when((F.col(ref).isNotNull()) & (F.col(cnt_zero) == n - 1), F.lit(-99997))
                     .otherwise(F.col(raw_metric)))

            # razões
            for metric in ["mean", "min", "max"]:
                denom = f"{var}_{metric}_{n}"
                ratio = f"{var}_ratio_ref_{metric}_{n}"
                generated_cols.append(ratio)

                df = df.withColumn(ratio,
                    F.when(F.col(denom) <= 0, F.col(denom))
                    .otherwise(F.col(ref) / F.col(denom)))

    # =========================
    # CONTROLE DE SAÍDA
    # =========================
    if not keep_intermediate:
        final_cols = list(dict.fromkeys(
            original_cols + generated_cols))
        df = df.select(final_cols)

    return df

In [ ]:
def features_de_tendencia_cat(df, flags, id_col="msno", time_col="safra", windows=[3, 6]):
    for flag in flags:
        df = df.withColumn(f"{flag}_raw", F.col(flag))

        for n in windows:
            w = Window.partitionBy(id_col).orderBy(time_col)

            lags = []
            for i in range(n):
                lag_col = f"{flag}_lag_{i}"
                df = df.withColumn(lag_col, F.lag(flag, i).over(w))
                lags.append(lag_col)

            arr = f"array({','.join(lags)})"

            df = (df
                .withColumn(f"{flag}_sum_{n}",
                    F.expr(f"aggregate(filter({arr}, x -> x is not null), 0, (acc, x) -> acc + x)"))
                .withColumn(f"{flag}_mean_{n}", F.col(f"{flag}_sum_{n}") / F.lit(n))
                .withColumn(f"{flag}_max_{n}", F.expr(f"array_max(filter({arr}, x -> x is not null))")))

    return df

#### Objetivo geral
O objetivo desta função é capturar **comportamentos temporais dos usuários**, indo além do valor pontual do mês de referência.  
Para cada cliente (`msno`) e safra (`yyyyMM`), são construídas variáveis que representam:
- o **nível atual** de uma métrica,
- o **histórico recente** dessa métrica,
- e a **relação entre o comportamento atual e o passado**.

Essa abordagem permite ao modelo identificar:
- padrões de crescimento ou queda,
- usuários novos vs recorrentes,
- mudanças abruptas de comportamento,
- ausência estrutural de informação.

---

#### Estrutura geral da função
A função recebe:
- uma lista de variáveis contínuas (ex: `num_100`, `total_secs`, `avg_secs_per_play`);
- janelas temporais fixas (`n = 3` e `n = 6` meses);
- e gera automaticamente **todas as transformações** para todas as variáveis listadas.

Para cada variável e cada janela temporal, são criados:
1. valores defasados (lags),
2. métricas estatísticas históricas,
3. razões entre o valor atual e o histórico,
4. versões com e sem códigos sentinela,
5. mantendo sempre o valor original (raw).

---

#### Lags temporais
Para cada variável `X`, são criados:
- `X_lag_0`: valor no mês de referência
- `X_lag_1`: valor no mês anterior
- ...
- `X_lag_(n-1)`: valor n−1 meses atrás

Esses lags formam a base para todas as transformações seguintes.

---

#### Variáveis `*_raw`
As variáveis com sufixo `_raw` representam o **valor matemático puro**, sem qualquer regra semântica adicional.

Exemplos:
- `num_100_raw`
- `num_100_mean_6_raw`
- `total_secs_max_3_raw`

Características:
- ignoram valores nulos quando possível;
- não utilizam códigos sentinela;
- podem assumir valores nulos ou zero naturalmente.

Uso principal:
- regressão linear,
- análises estatísticas,
- inspeção e debug.

---

#### Métricas históricas criadas
Para cada variável `X` e janela `n`, são criadas:

- `X_mean_n_raw`: média dos últimos `n` meses
- `X_min_n_raw`: mínimo dos últimos `n` meses
- `X_max_n_raw`: máximo dos últimos `n` meses

Essas métricas descrevem o **nível típico**, o **pior cenário** e o **melhor cenário recente** do usuário.

---

#### Variáveis sem `_raw` (com semântica)
As variáveis sem o sufixo `_raw` aplicam **regras semânticas** para diferenciar situações estruturalmente distintas, que numericamente poderiam parecer iguais.

Essas variáveis utilizam **códigos sentinela negativos**, que carregam significado comportamental:

| Código | Significado |
|------|-------------|
| `-99998` | Não há registros em nenhum dos últimos `n` meses |
| `-99999` | Todos os valores nos últimos `n` meses são zero |
| `-99995` | Valor atual é zero, mas havia histórico válido |
| `-99996` | Valor atual existe, mas histórico é todo nulo |
| `-99997` | Valor atual existe, mas histórico é todo zero |

Esses códigos permitem ao modelo (especialmente árvores):
- distinguir ausência de uso vs ausência de dados,
- identificar usuários novos,
- detectar interrupções de comportamento.

---

#### Razões (variáveis de mudança de comportamento)
Para cada métrica histórica, são criadas razões do tipo:

- `X_ratio_ref_mean_n`
- `X_ratio_ref_min_n`
- `X_ratio_ref_max_n`

Essas variáveis comparam o valor atual (`lag_0`) com o histórico recente.

Interpretação:
- valor ≈ 1 → comportamento estável
- valor > 1 → aumento recente
- valor < 1 → queda recente

Caso o denominador seja inválido (zero, nulo ou sentinela), a razão herda o código sentinela, evitando divisões inválidas.

---

#### Separação entre estatística e semântica
Um ponto central do design é **não misturar estatística com semântica**.

Por isso:
- variáveis `_raw` são mantidas limpas e contínuas;
- variáveis sem `_raw` carregam contexto comportamental.

Essa separação permite:
- uso seguro em regressão linear (via `_raw`);
- exploração rica de padrões em modelos baseados em árvore (via sentinelas).

---

#### Variáveis de flags (binárias)
Para variáveis categóricas/binárias (ex: `is_cancel`, `is_auto_renew`), são criadas transformações específicas:

- `flag_sum_n`: número de ocorrências nos últimos `n` meses
- `flag_mean_n`: frequência relativa
- `flag_max_n`: ocorreu pelo menos uma vez

Essas variáveis capturam **persistência, recorrência e reincidência** de eventos.

---

#### Benefícios da abordagem
Essa arquitetura de features:
- reduz dependência de um único mês,
- melhora estabilidade temporal,
- captura mudanças de comportamento,
- é robusta a dados faltantes,
- funciona bem tanto para regressão quanto para árvores.

Além disso, permite seleção posterior de variáveis sem refazer o pipeline, mantendo rastreabilidade e flexibilidade.

---

#### Observação final
Nem todas as variáveis criadas necessariamente entrarão no modelo final.  
A função foi desenhada para **gerar um espaço rico de candidatos**, permitindo que a seleção seja feita de forma empírica, baseada em validação e performance.
